# Yastn MPS generator for Lindbladian

## Structure

### Input

#### 1. Latex String (kw: ltx_str)
The Lindbladian as a Latex string in some form<br>
e.g.
$$
-i {\sum_{j=0}^{N-1} ([\sigma_{j}^{z}, \rho])} 
+ {\sum_{i,j = 0}^{N-1} \gamma_{i,j} (\sigma_{i}^{z} \rho \sigma_{j}^{z} - \frac{1}{2} \{ \sigma_{j}^{z} \sigma_{i}^{z}, \rho \} )}
$$
or
$$
-i {\sum_{j=0}^{N-1} (\sigma_{j,ket}^{z} - \sigma_{j,bra}^{z})} 
+ {\sum_{i,j = 0}^{N-1} \gamma_{i,j} (\sigma_{i,ket}^{z} \sigma_{j,bra}^{z} - \frac{1}{2} ( \sigma_{j,ket}^{z} \sigma_{i,ket}^{z} + \sigma_{j,bra}^{z} \sigma_{i,bra}^{z} ) )}
$$

#### 2. Parameters (kw: params)
Any parameters that occur in the Latex string are defined here. <br>
<br>
e.g. <br>
{"t": t, <br>
"mu": mu, <br>
"sites": list(range(N)), <br>
"NN": [(i, i+1) for i in range(N-1)]}

### Output
A yastn.Mps object is returned

### Procedure
Perform a series of translation of latex formulas until the _latex2term can handle it while expanding the formula and parameters accordingly during the process <br>
#### Start
with
$$
-i \sum_{j=0}^{N-1} ([\sigma_{j}^{z}, \rho])
+ \sum_{i,j = 0}^{N-1} \gamma_{i,j} (\sigma_{i}^{z} \rho \sigma_{j}^{z} - \frac{1}{2} \{ \sigma_{j}^{z} \sigma_{i}^{z}, \rho \} )
$$
resolve commutators
$$
-i \sum_{j=0}^{N-1} (\sigma_{j}^{z}\rho - \rho\sigma_{j}^{z})
+ \sum_{i,j = 0}^{N-1} \gamma_{i,j} (\sigma_{i}^{z} \rho \sigma_{j}^{z} - \frac{1}{2} ( \sigma_{j}^{z} \sigma_{i}^{z}\rho + \rho\sigma_{j}^{z} \sigma_{i}^{z} ) )
$$
extract $\rho$
$$
-i \sum_{j=0}^{N-1} (\sigma_{j,ket}^{z} - \sigma_{j,bra}^{z})
+ \sum_{i,j = 0}^{N-1} \gamma_{i,j} (\sigma_{i,ket}^{z} \sigma_{j,bra}^{z} - \frac{1}{2} ( \sigma_{j,ket}^{z} \sigma_{i,ket}^{z} + \sigma_{j,bra}^{z} \sigma_{i,bra}^{z} ) )
$$
rephrase summation
$$
-i \sum_{j,jk,jb \in 1range0toN-1} (\sigma_{j,ket}^{z} - \sigma_{j,bra}^{z})
+ \sum_{i,j,ij,ib,jk,jb \in 2range0toN-1} \gamma_{i,j} (\sigma_{i,ket}^{z} \sigma_{j,bra}^{z} - \frac{1}{2} ( \sigma_{j,ket}^{z} \sigma_{i,ket}^{z} + \sigma_{j,bra}^{z} \sigma_{i,bra}^{z} ) )
$$
rename (greek) symbols (modify parameters)
$$
-i \sum_{j,jk,jb \in 1range0toN-1} (z_{j,ket} - z_{j,bra})
+ \sum_{i,j,ij,ib,jk,jb \in 2range0toN-1} gamma_{i,j} (z_{i,ket} z_{j,bra} - \frac{1}{2} ( z_{j,ket} z_{i,ket} + z_{j,bra} z_{i,bra} ) )
$$
extract constants (added to params)
$$
-imun \sum_{j,jk,jb \in 1range0toN-1} (z_{j,ket} - z_{j,bra})
+ \sum_{i,j,ij,ib,jk,jb \in 2range0toN-1} gamma_{i,j} (z_{i,ket} z_{j,bra} - 1div2 ( z_{j,ket} z_{i,ket} + z_{j,bra} z_{i,bra} ) )
$$
resolve ket/bra-marked operators by removing notation and complex conjugating bra-marked ops (and adding respective operators to parameters)
$$
-imun \sum_{j,jk,jb \in 1range0toN-1} (z_{jk} - zcc_{jb})
+ \sum_{i,j,ij,ib,jk,jb \in 2range0toN-1} gamma_{i,j} (z_{ik} zcc_{jb} - 1div2 ( z_{jk} z_{ik} + zcc_{jb} zcc_{ib} ) )
$$
where with the help of a a cConjugate(op) method $zcc_j = cConjugate(z)_j = z_j^*$. <br>
<br>
which should then yield the respective output through the latex2term-, term2HTerm- and generate_mpo-methods




In [ ]:
import numpy as np
import yastn
from ..src.assets.tn.mps import _generator_class as gen_mps
import yastn.tn.mps as mps_fun
import yastn.operators._spin12 as spin_ops
from ..src.assets.tn.mps import _env as env

ImportError: attempted relative import with no known parent package

In [1]:
config_kwargs = {"backend": "np"}
sym = 'dense'
N = 4

ops = spin_ops.Spin12(sym=sym, **config_kwargs)

ltx_str_full = r"-i (\sum_{j=0}^{N-1} ([\sigma_j^z, \rho])) + \sum_{j,k = 0}^{N-1} \gamma_{j,k} (\sigma_{j}^{z} \rho \sigma_{k}^{z} - \frac{1}{2} \{ \sigma_{k}^{z} \sigma_{j}^{z}, \rho \} )"
ltx_str_no_com = r"-i {\sum_{j=0}^{N-1} (\sigma_{j,ket}^{z} - \sigma_{j,bra}^{z})} + {\sum_{j,k = 0}^{N-1} \gamma_{j,k} (\sigma_{j,ket}^{z} \sigma_{k,bra}^{z} - \frac{1}{2} ( \sigma_{k,ket}^{z} \sigma_{j,ket}^{z} + \sigma_{k,bra}^{z} \sigma_{j,bra}^{z} ) )}"
ltx_str_simple = r"-imun (\sum_{j,jk,jb \in Nx} (z_{jk} - zcc_{jb})) + \sum_{j,k,jk,jb,kk,kb \in NxN} gamma_{j,k} (z_{j} zcc_{k} - 1.0div2.0 ( z_{k} z_{j} + zcc_{k} zcc_{j} ) )"
parameters = {"\\gamma": np.ones([4,4]),
              "N": N,
              "imun": 1j,
              "1.0div2.0": 1/2,
              "Nx": [(i,2*i,2*i+1) for i in range(N)],
              "NxN": [(i,j,2*i,2*i+1,2*j,2*j+1) for i in range(N) for j in range(N)]
              }

generate = gen_mps.GenericGenerator(2*N, ops, debug=False)
mpo = generate.mpo_from_latex(ltx_str_simple, parameters=parameters, ignore_i=False, rho2ketbra=True)

mpo = generate.mpo_from_latex(ltx_str_no_com, parameters=parameters, ignore_i=False, rho2ketbra=True)

mpo = generate.mpo_from_latex(ltx_str_full, parameters=parameters, ignore_i=False, rho2ketbra=True)
    

NameError: name 'spin_ops' is not defined

# Yastn Environment for Lindbladian
%load_ext jupyter_tikz

## Structure (Initialization)
%%tikz
\begin{tikzpicture}
    \node[rectangle, draw] (0) {$F[-1,0]$}
    \node[draw, shape=circle] (1) {$A_{j}^{\dag}$};
    \node[draw, shape=circle] (2) [below left of=1] {$A_{j}$};
    \node[draw, shape=circle] (3) [below of=2] {$L_{j,ket}$};
    \node[draw, shape=circle] (4) [right of=3] {$L_{j,bra}^{\dag}$};
    \node[draw, shape=circle] (5) [below of=3] {$A_{j}$};
    \node[draw, shape=circle] (6) [below right of=5] {$A_{j}^{\dag}$};
    \node[rectangle, draw] (7) {$F[N,N+1]$}

    \draw[->] (1) to [out=90, in=270, looseness=2.5] (2)
    \draw[->] (2) to (3)
    \draw[--] (3) to (4)
    \draw[->] (3) to (5)
    \draw[->] (5) to [out=270, in=90, looseness=2.5] (6)
    \draw[->] (6) to (4)
    \draw[->] (4) to (1)
    
    \draw (0 |- 0.east) -- (1)
    (0 |- 0.east) -- (2)
    \draw [thick] (0 |- 0.east) -- (3)
    \draw (0 |- 0.east) -- (5)
    (0 |- 0.east) -- (6)
    
    \draw (1) -- (1 |- 7.west)
    (2) -- (2 |- 7.west)
    \draw [thick] (4) -- (4 |- 7.west)
    \draw (5) -- (5 |- 7.west)
    (6) -- (6 |- 7.west)
\end{tikzpicture}


## Update (Contraction)